<a href="https://colab.research.google.com/github/vitaldb/examples/blob/master/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction Model for Hepatocellular cardinoma in Hepatitis B patients

## Upgrade XGboost library

In [6]:
!pip uninstall xgboost -y
!pip install xgboost

Found existing installation: xgboost 1.6.2
Uninstalling xgboost-1.6.2:
  Successfully uninstalled xgboost-1.6.2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached xgboost-1.6.2-py3-none-manylinux2014_x86_64.whl (255.9 MB)


## Download model and sample files

In [12]:
!wget https://github.com/vitaldb/hbvhcc/raw/main/model.json
!wget https://github.com/vitaldb/hbvhcc/raw/main/sample.csv

--2022-11-24 15:29:46--  https://github.com/vitaldb/hbvhcc/raw/main/model.json
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/vitaldb/planb/raw/main/model.json [following]
--2022-11-24 15:29:47--  https://github.com/vitaldb/planb/raw/main/model.json
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/vitaldb/planb/main/model.json [following]
--2022-11-24 15:29:47--  https://raw.githubusercontent.com/vitaldb/planb/main/model.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60814 (59K) [text/plain]
Saving 

## Simple example for single patient



In [13]:
import xgboost as xgb
import numpy as np

# sample values
male = 0
age = 48
e_t = 1
lc = 1  # None for missing value
plt = 172
alb = 4.1
tbil = 1.1
alt = 122
dna = 74279943
hbeag = 1

model = xgb.Booster()
model.load_model('model.json')
x = np.array([male, age, e_t, lc, plt, alb, tbil, alt, dna, hbeag]).astype(float)
x = x[None, ...]
x = xgb.DMatrix(x)
y = model.predict(x)[0]

print('score = {:.3f}'.format(y))

score = 0.196


## Example code using CSV file for multiple patients 

In [14]:
import xgboost as xgb
import numpy as np
import pandas as pd

model = xgb.Booster()
model.load_model('model.json')

df = pd.read_csv('sample.csv')
x = df[['male', 'age', 'e_t', 'lc', 'plt', 'alb', 'tbil', 'alt', 'dna', 'hbeag']].astype(float).values
df['pred'] = model.predict(xgb.DMatrix(x))
df.to_csv('prediction.csv')
print(df)

      no  male   age  e_t   lc    plt  alb  tbil    alt          dna  hbeag  \
0    1.0   1.0  33.0  0.0  0.0  167.0  4.4   1.1   84.0  170000000.0    1.0   
1    2.0   0.0  56.0  0.0  0.0  168.0  3.6   0.7  223.0    7990000.0    1.0   
2    3.0   0.0  62.0  1.0  1.0  164.0  4.3   0.5  104.0       6080.0    0.0   
3    4.0   1.0  46.0  1.0  0.0  317.0  4.3   0.7   66.0      45100.0    0.0   
4    5.0   1.0  54.0  1.0  1.0  122.0  4.6   1.3  147.0   12500000.0    0.0   
5    6.0   1.0  30.0  0.0  0.0  198.0  4.4   1.4  955.0  170000000.0    1.0   
6    7.0   1.0  48.0  0.0  1.0  111.0  4.2   1.3   95.0     262000.0    0.0   
7    8.0   0.0  58.0  1.0  1.0  138.0  4.3   0.7   45.0      17100.0    0.0   
8    9.0   0.0  50.0  0.0  1.0  154.0  4.2   1.0   61.0     790000.0    0.0   
9   10.0   0.0  65.0  0.0  0.0  187.0  4.6   0.6   23.0          0.0    0.0   
10   NaN   NaN   NaN  NaN  NaN    NaN  NaN   NaN    NaN          NaN    NaN   

        pred  
0   0.095564  
1   0.163563  
2   0.